In [4]:
import json
import pandas as pd

#load the jason file
file_path = r"C:\Users\sajad\Downloads\2armed_bandit.json"
with open(file_path, "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.head(204)






,rt,stimulus,response,trial_type,trial_index,time_elapsed,internal_node_id,bean_trial_duration,bean_duration,bean_stimulus,...,bean_text,bean_color,bean_correct,value,bean_html_array,bean_values,bean_time_after_response,bean_bandits,bean_value,bean_score
0,6946.0,<div style='color:white'>Welcome to our decisi...,,html-keyboard-response,0,6951,0.0-0.0-0.0,NaN,NaN,<div style='color:white'>Welcome to our decisi...,...,Welcome to our decision-making experiment. In ...,white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1287.0,"<div style='color:white'>\n <div class=""slo...",,html-keyboard-response,1,8241,0.0-0.0-1.0,NaN,NaN,"<div style='color:white'>""\n <div class=""sl...",...,"""\n <div class=""slotmachine"" style=""positio...",white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1130.0,<div style='color:white'>After selecting a ban...,,html-keyboard-response,2,9372,0.0-0.0-2.0,NaN,NaN,<div style='color:white'>After selecting a ban...,...,"After selecting a bandit, you will receive fee...",white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,520.0,<div style='color:white'>Your goal is to earn ...,,html-keyboard-response,3,9894,0.0-0.0-3.0,NaN,NaN,<div style='color:white'>Your goal is to earn ...,...,Your goal is to earn as many points as possibl...,white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1414.0,"[<div class=""slotmachine"" style=""position: abs...",0,html-choice,4,13317,0.0-1.0-0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,"[<div class=""slotmachine"" style=""position: abs...","[0, 1]",2000.0,"[{'color': 'orange', 'value': 0}, {'color': 'b...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,NaN,<div style='color:white'>Score: 63</div>,None,html-keyboard-response,199,374656,0.0-1.0-1.97,1000.0,1000.0,<div style='color:white'>Score: 63</div>,...,Score: 63,white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,1108.0,"[<div class=""slotmachine"" style=""position: abs...",0,html-choice,200,377776,0.0-1.0-0.98,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,"[<div class=""slotmachine"" style=""position: abs...","[0, 0]",2000.0,"[{'color': 'orange', 'value': 0}, {'color': 'b...",0.0,63.0
201,NaN,<div style='color:white'>Score: 63</div>,None,html-keyboard-response,201,378780,0.0-1.0-1.98,1000.0,1000.0,<div style='color:white'>Score: 63</div>,...,Score: 63,white,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,610.0,"[<div class=""slotmachine"" style=""position: abs...",0,html-choice,202,381398,0.0-1.0-0.99,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,"[<div class=""slotmachine"" style=""position: abs...","[1, 1]",2000.0,"[{'color': 'orange', 'value': 1}, {'color': 'b...",1.0,64.0


In [5]:
import pandas as pd

#keep only the rows from 4 to 204 which are the rows of the actual trials
df = df.iloc[4:204].copy()

#keep only essential columns
df = df[['trial_index', 'response', 'bean_value', 'bean_bandits']]

#rename the columns
df.rename(columns={
    'trial_index': 'trial',
    'response': 'choice',
    'bean_value': 'reward',
    'bean_bandits': 'bandit_info'
    }, inplace=True)

#reset the index
df.reset_index(drop=True, inplace=True)

print(df.head())







   trial choice  reward                                        bandit_info
0      4      0     0.0  [{'color': 'orange', 'value': 0}, {'color': 'b...
1      5   None     NaN                                                NaN
2      6      1     1.0  [{'color': 'orange', 'value': 1}, {'color': 'b...
3      7   None     NaN                                                NaN
4      8      1     1.0  [{'color': 'orange', 'value': 0}, {'color': 'b...


In [8]:
#our one-state env
import numpy as np

class OneStepBanditEnv:
    def __init__(self, p_bandit0 = 0.8,             #probability of getting reward from the first bandit
                 p_bandit1 = 0.2,                   #robability of getting reward from the second bandit
                 reversal_trial = 50,               #changing the probabilities in after the half of the trials
                 n_trials = 100):
        self.p0_init = p_bandit0
        self.p1_init = p_bandit1
        self.reversal_trial = reversal_trial
        self.n_trials = n_trials
        self.trial = 0
        #current probabilities:
        self.p0 = p_bandit0
        self.p1 = p_bandit1
        self.done = False

    def reset(self):
        self.trial = 0
        self.p0 = self.p0_init
        self.p1 = self.p1_init
        self.done = False
        return None             #single-state env, so no real "state" to return
    
    def step(self, action):
        """
        action: 0 or 1 for bandit choice
        returns: reward, done, info
        """
        #revers the probs when in the correct trial
        if self.reversal_trial is not None and self.trial == self.reversal_trial:
            old_p0 = self.p0
            self.p0 = self.p1
            self.p1 = old_p0

        #get reward from chosen bandit
        if action == 0:
            reward = np.random.binomial(1, self.p0)
        elif action== 1:
            reward = np.random.binomial(1, self.p1)
        else:
            raise ValueError("action must be 0 or 1")
        
        self.trial += 1
        done = (self.trial >= self.n_trials)      #done after N_trials
        self.done = done
        return reward, done, {}
        

In [9]:
def softmax(vals, beta):
    """
    Softmax function for action selection
    vals: The array of action values
    beta: Inverse temparature parameter for the softmax policy (higher beta -> more deterministic)
    return: The probabilities of each action, sums up to 1
    """

    """Compute softmax values for each sets of scores in x."""

    result = np.exp(vals * beta) / np.sum(np.exp(vals * beta))

    return result

In [10]:
class BaseAgent:
    def __init__(self, alpha = 0.1, beta = 1.0):
        """
        a base agent for a single-state 2-bandit task
        alpha: learning rate
        beta: inverse temprature
        """
        self.alpha = alpha
        self.beta = beta
        self.Q = np.zeros(2)        #store Q for each action(2 actions)

    def get_action_probs(self):
        """
        return softmax probabilities for two bandits
        """
        probs = softmax(self.Q, self.beta)
        return probs
    
    def policy(self):
        """
        choose an action using the softmax policy
        """
        probs = self.get_action_probs()
        action = np.random.choice([0,1], p= probs)
        return action
    
    def update_values(self, action, reward):
        """
        update the choosen action values using the Rescorla-Wagner rule
        """

        rpe = reward - self.Q[action]
        self.Q[action] += self.alpha * rpe

In [11]:
class ModelFreeAgent(BaseAgent):
    def calculate_rpe(self, state, action, reward):
        rpe = reward - self.q_table[state, action]
        return rpe

In [13]:
#Thompson Sampling Agent with no decay
import numpy as np

class ThompsonSamplingAgentNoDecay(BaseAgent):
    def __init__(self, alpha0 = 0.1, beta0 = 1.0):
        """
        alpha0 and beta0: initial Beta prior hyperparamrters
        """
        self.alpha0 = alpha0
        self.beta0 = beta0
        self.reset()

    def reset(self):
        self.alphas = [self.alpha0, self.alpha0]
        self.betas = [self.beta0, self.beta0]

    def choose_action(self):
        #sample from each arm's Beta distribution
        sample_0 = np.random.beta(self.alphas[0], self.betas[0])
        sample_1 = np.random.beta(self.alphas[1], self.betas[1])
        return 0 if sample_0 > sample_1 else 1
    
    def update(self, action, reward):
        #update the chosen arm's posterior
        if reward == 1:
            self.alphas[action] += 1
        else:
            self.betas[action] += 1


In [14]:
#Thompson Sampling Agent with decay
import numpy as np

class ThompsonSamplingAgentDecay(BaseAgent):
    def __init__(self, alpha0 = 1.0, beta0 = 1.0, decay = 0.97):
        """
        alpha0 and beta0: initial Beta prior hyperparamrters
        decay: if <1 we decay alpha and beta after each trial for forgetting
        and this helps the model to adapt to change in probabilities in the task
        """
        self.alpha0 = alpha0
        self.beta0 = beta0
        self.decay = decay
        self.reset()

    def reset(self):
        self.alphas = [self.alpha0, self.alpha0]
        self.betas = [self.beta0, self.beta0]

    def choose_action(self):
        #sample from each arm's Beta distribution
        sample_0 = np.random.beta(self.alphas[0], self.betas[0])
        sample_1 = np.random.beta(self.alphas[1], self.betas[1])
        return 0 if sample_0 > sample_1 else 1
    
    def update(self, action, reward):
        #decay the existing posterior values
        self.alphas[action] *= self.decay
        self.betas[action] *= self.decay

        #update the chosen arm's posterior
        if reward == 1:
            self.alphas[action] += 1
        else:
            self.betas[action] += 1

